# MLOps Steam


Buenas y bienvenidos a este Notebook donde haremos el proceso de ETL a 3 datasets brindados por la plataforma de juegos Steam donde nosotros podremos practicar y brindar una solucion al problema que estan teniendo. Una vez que tratemos los datos nuestro objetivo sera hacer un analisis exploratorio de los datos y a raiz de esto sacar un modelo funcional de inteligencia artificial, que podra ser consumida desde una api por Render.


Comenzemos con la lectura de los datos y la limpieza de los mismos.

In [102]:
#instalamos todas las librerias necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import ast

In [103]:
import json

data = []
with open('data/output_steam_games.json', 'r') as f:
    for line in f:
        try:
            obj = json.loads(line)
            data.append(obj)
        except json.JSONDecodeError as e:
            print("Error en línea:", line)

# Convierte la lista de objetos en un DataFrame
steam = pd.DataFrame(data)

# Imprime el DataFrame
print(steam.shape)
steam.head()


(120445, 13)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
steam = steam.dropna(thresh=3)
print(steam.shape)
steam.head(3)

(32135, 13)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com


In [105]:
rows = []
with open('data/australian_users_items.json', 'r', encoding='UTF-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

In [106]:
user_items = pd.DataFrame(rows)

In [107]:
user_items = user_items.dropna(thresh=3)
print(user_items.shape)
user_items.head(3)

(88310, 5)


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [108]:
rows = []
with open('data/australian_user_reviews.json', 'r', encoding='UTF-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

In [109]:
user_reviews = pd.DataFrame(rows)

In [110]:
user_reviews = user_reviews.dropna(thresh=3)
print(user_reviews.shape)
user_reviews.head(3)


(25799, 3)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


### En este momento ya poseemos los 3 dataframes necesarios para comenzar a trabajar a responder las preguntas solicitadas, vamos a ello una por una


In [129]:
endpoint1 = steam[['id', 'title', 'price']]
endpoint1

,id,title,price
88310,761140,Lost Summoner Kitty,4.99
88311,643980,Ironbound,Free To Play
88312,670290,Real Pool 3D - Poolians,Free to Play
88313,767400,弹炸人2222,0.99
88314,773570,NaN,2.99
...,...,...,...
120440,773640,Colony On Mars,1.99
120441,733530,LOGistICAL: South Africa,4.99
120442,610660,Russian Roads,1.99
120443,658870,EXIT 2 - Directions,4.99


277

In [233]:
items_ids = []
lista_items = user_items[user_items['user_id'] == 'evcentric']['items']
for i in lista_items:
    print(i)


[{'item_id': '1200', 'item_name': 'Red Orchestra: Ostfront 41-45', 'playtime_forever': 923, 'playtime_2weeks': 0}, {'item_id': '1230', 'item_name': 'Mare Nostrum', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '1280', 'item_name': "Darkest Hour: Europe '44-'45", 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '1520', 'item_name': 'DEFCON', 'playtime_forever': 158, 'playtime_2weeks': 0}, {'item_id': '220', 'item_name': 'Half-Life 2', 'playtime_forever': 1323, 'playtime_2weeks': 0}, {'item_id': '320', 'item_name': 'Half-Life 2: Deathmatch', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '340', 'item_name': 'Half-Life 2: Lost Coast', 'playtime_forever': 90, 'playtime_2weeks': 0}, {'item_id': '360', 'item_name': 'Half-Life Deathmatch: Source', 'playtime_forever': 0, 'playtime_2weeks': 0}, {'item_id': '380', 'item_name': 'Half-Life 2: Episode One', 'playtime_forever': 234, 'playtime_2weeks': 0}, {'item_id': '400', 'item_name': 'Portal', 'playtime_forever': 1

In [206]:
"""
def userdata( User_id : str ): Debe devolver cantidad de dinero gastado por el usuario,
 el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
"""
def userdata(id):
    user = user_items[user_items['user_id'] == id]
    iteraciones = user['items_count']
    items = user['items']
    
    

In [207]:
userdata('evcentric')

2    [{'item_id': '1200', 'item_name': 'Red Orchest...
Name: items, dtype: object